In [54]:
import numpy as np
import pandas as pd
import glob, os, random, shutil, sys, time
from scipy import stats
from matplotlib import pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

import sys

#mac
util_path = "/Users/kimd999/research/projects/Katrina/srpAnalytics/code/latest/util"

#constance
'''args = sys.argv[0:]
py_file = args[0]

code_location = os.path.dirname(os.path.abspath(py_file))
index_of_latest = code_location.index('latest')
util_path = os.path.join(code_location[:index_of_latest], "latest", "util")
'''

sys.path.insert(0, util_path)

# These two lines will automatically reload updated modules of mine
# (therefore, I don't need to restart kernel after editing importing modules)
#%load_ext autoreload
#%autoreload 2

In [6]:
import generate_dose_response as gdr
import BMD_BMDL_estimation as bmdest
import Plot_Save as ps

In [4]:
starting_dir = os.getcwd()
print (starting_dir)

/Users/kimd999/research/projects/Katrina/srpAnalytics/code/latest/2_bmd/2_analysis/Lindsay


In [96]:
#complete_file_path = '/Users/kimd999/research/projects/Katrina/per_each_data/Lindsay/new/final_input.csv'
complete_file_path = '/Users/kimd999/research/projects/Katrina/per_each_data/Lindsay/new/input/each_chemical/di-n-butyl_phthalate.csv'
df_test_dose_response = pd.read_csv(complete_file_path, header = 0)
pd.set_option('display.max_columns', None)
display(df_test_dose_response.head())
#display(np.unique(morpho_data.well))

,chemical.id,dose,total_num,num_affected
0,di-n-butyl phthalate,0.0,36,2
1,di-n-butyl phthalate,2.0,36,2
2,di-n-butyl phthalate,3.0,36,3
3,di-n-butyl phthalate,4.0,36,8
4,di-n-butyl phthalate,4.5,36,19


In [97]:
full_devel = "full"
#full_devel = "devel"

if (full_devel == "full"):
    chemical_id_from_here = np.unique(df_test_dose_response['chemical.id'])
else: # full_devel = "devel"
    #chemical_id_from_here = ['Benzophenone']
    chemical_id_from_here = ['Benzyl butyl phthalate']
    
start_time = time.time()

for chemical_id in chemical_id_from_here:
    print("chemical_id:" + str(chemical_id))
    
    # subset original dataframe for a user-specified chemical and end_point pair
    df_test_dose_response_per_chemical = df_test_dose_response.loc[df_test_dose_response['chemical.id'] == chemical_id,['chemical.id', 'dose', 'total_num', 'num_affected']]
    print (f"df_test_dose_response_per_chemical:\n{df_test_dose_response_per_chemical}")
    
    # Fit dose response models
    df_test_dose_response_per_chemical = df_test_dose_response_per_chemical.sort_values(by=['dose'])    
    print (f"df_test_dose_response_per_chemical:\n{df_test_dose_response_per_chemical}")

    try:
        model_predictions = bmdest.analyze_dose_response_data(df_test_dose_response_per_chemical)
        print (f"model_predictions:\n{model_predictions}")
    except:
        continue
    
    # Select best model
    selected_model_params = bmdest.select_model(model_predictions)
    print (selected_model_params)
    
    end_point = "Lindsay"
    bmd_feasibility_flag = 2

    # Check if unique model is found
    unique_model_flag = selected_model_params['no_unique_model_found_flag']

    if(unique_model_flag == 0):
        # Generate report
        #print(df_test_dose_response.dose[-1:])

        ps.save_results_good_data_unique_model(df_test_dose_response_per_chemical, bmd_feasibility_flag, model_predictions, selected_model_params, str(chemical_id), end_point)
    else:
        bmd_analysis_flag = selected_model_params['model_select_flag']
        if(bmd_analysis_flag == 1):
            ps.save_results_poor_data_or_no_convergence(df_test_dose_response_per_chemical, bmd_feasibility_flag, str(chemical_id), end_point, selected_model_params)
        else:
            ps.save_results_good_data_nounique_model(df_test_dose_response_per_chemical, bmd_feasibility_flag, model_predictions, selected_model_params, str(chemical_id), end_point)

end_time = time.time()
time_took = str(round((end_time-start_time), 1)) + " seconds"
print ("BMD calculation is done, it took:"+str(time_took)) 

chemical_id:di-n-butyl phthalate
df_test_dose_response_per_chemical:
            chemical.id  dose  total_num  num_affected
0  di-n-butyl phthalate   0.0         36             2
1  di-n-butyl phthalate   2.0         36             2
2  di-n-butyl phthalate   3.0         36             3
3  di-n-butyl phthalate   4.0         36             8
4  di-n-butyl phthalate   4.5         36            19
5  di-n-butyl phthalate   5.0         36            21
6  di-n-butyl phthalate   8.0         36            36
7  di-n-butyl phthalate  10.0         36            36
df_test_dose_response_per_chemical:
            chemical.id  dose  total_num  num_affected
0  di-n-butyl phthalate   0.0         36             2
1  di-n-butyl phthalate   2.0         36             2
2  di-n-butyl phthalate   3.0         36             3
3  di-n-butyl phthalate   4.0         36             8
4  di-n-butyl phthalate   4.5         36            19
5  di-n-butyl phthalate   5.0         36            21
6  di-n-butyl p

In [ ]:
a=b

In [66]:
start_time = time.time()

for csv_file in glob.glob("*.csv"):
    print (f"\ncsv_file:{csv_file}")
    df_test_dose_response_per_chemical = pd.read_csv(csv_file, header = 0)
    pd.set_option('display.max_columns', None)
#    display(df_test_dose_response.head())
    model_predictions = bmdest.analyze_dose_response_data(df_test_dose_response_per_chemical)
    #print (f"model_predictions:\n{model_predictions}")

    # Select best model
    selected_model_params = bmdest.select_model(model_predictions)
    #print (selected_model_params)
    
    end_point = "Lindsay"
    bmd_feasibility_flag = 2

    # Check if unique model is found
    unique_model_flag = selected_model_params['no_unique_model_found_flag']

    if(unique_model_flag == 0):
        # Generate report
        #print(df_test_dose_response.dose[-1:])

        ps.save_results_good_data_unique_model(df_test_dose_response_per_chemical, bmd_feasibility_flag, model_predictions, selected_model_params, str(chemical_id), end_point)
    else:
        bmd_analysis_flag = selected_model_params['model_select_flag']
        if(bmd_analysis_flag == 1):
            ps.save_results_poor_data_or_no_convergence(df_test_dose_response_per_chemical, bmd_feasibility_flag, str(chemical_id), end_point, selected_model_params)
        else:
            ps.save_results_good_data_nounique_model(df_test_dose_response_per_chemical, bmd_feasibility_flag, model_predictions, selected_model_params, str(chemical_id), end_point)

end_time = time.time()
time_took = str(round((end_time-start_time), 1)) + " seconds"
print ("BMD calculation is done, it took:"+str(time_took)) 


csv_file:fit_vals_2021_12_02.csv


KeyError: "None of [Index(['dose', 'num_affected', 'total_num'], dtype='object')] are in the [columns]"

In [62]:
# divide into csv files

full_devel = "full"
#full_devel = "devel"

if (full_devel == "full"):
    chemical_id_from_here = np.unique(df_test_dose_response['chemical.id'])
else: # full_devel = "devel"
    #chemical_id_from_here = ['Benzophenone']
    chemical_id_from_here = ['Benzyl butyl phthalate']
    
start_time = time.time()

for chemical_id in chemical_id_from_here:
    print("chemical_id:" + str(chemical_id))
    
    # subset original dataframe for a user-specified chemical and end_point pair
    df_test_dose_response_per_chemical = df_test_dose_response.loc[df_test_dose_response['chemical.id'] == chemical_id,['chemical.id', 'dose', 'total_num', 'num_affected']]
 #   print (f"df_test_dose_response_per_chemical:\n{df_test_dose_response_per_chemical}")
    
    # Fit dose response models
    df_test_dose_response_per_chemical = df_test_dose_response_per_chemical.sort_values(by=['dose'])    
#    print (f"df_test_dose_response_per_chemical:\n{df_test_dose_response_per_chemical}")

    output_filename = str(chemical_id) + ".csv"
    df_test_dose_response_per_chemical.to_csv(output_filename, index=False) 


chemical_id:Benzophenone
chemical_id:Benzyl butyl phthalate
chemical_id:Benzyl salicylate
chemical_id:Bis(2-ethylhexyl)phthalate
chemical_id:Butylated hydroxytoluene
chemical_id:DEET
chemical_id:Diethyl phthalate
chemical_id:Diisobutyl phthalate
chemical_id:Dinonyl phthalate
chemical_id:Galaxolide
chemical_id:Lilial
chemical_id:NHBE Mix
chemical_id:Tonalide
chemical_id:Triphenyl phosphate
chemical_id:di-n-butyl phthalate
